<a href="https://colab.research.google.com/github/francotestori/aauba_02/blob/master/armado_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Practico 2

## Armado del dataset

In [13]:
!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
!mkdir speechcommands
!tar -xf speech_commands_v0.01.tar.gz -C /content/speechcommands

--2020-06-29 23:00:50--  http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 209.85.200.128, 2607:f8b0:4001:c16::80
Connecting to download.tensorflow.org (download.tensorflow.org)|209.85.200.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1489096277 (1.4G) [application/gzip]
Saving to: ‘speech_commands_v0.01.tar.gz.2’

speech_commands_v0. 100%[===================>]   1.39G   165MB/s    in 8.4s    

2020-06-29 23:00:59 (169 MB/s) - ‘speech_commands_v0.01.tar.gz.2’ saved [1489096277/1489096277]

mkdir: cannot create directory ‘speechcommands’: File exists


In [45]:
# Import de Librerías
import librosa
import glob
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from IPython.display import Audio
from librosa.display import specshow

In [52]:
directorio='Particiones_grabadas'

In [53]:
# Busco y guardo en un mapa todos los audio files correspondientes a los digit_keys
digit_command_keys = [
  'zero',                    
  'one',
  'two',
  'three',
  'four',
  'five',
  'six',
  'seven',
  'eight',
  'nine'
]

audio_files = {}

for key in digit_command_keys:
    directory = f'./{directorio}/{key}/*.wav'
    audio_files[key] = glob.glob(directory)

In [54]:
# Utilizamos la función propuesta en el TP 
# para hacer la extracción de los datos 
# correspondientes a cada audio file

def calculate_features(
        filename,
        n_mfcc=12,
        delta=True,
        deltadelta=True,
        energy=True, 
        summary_fn = [np.mean, np.std], 
        summary_names=['mean','std']):
    #Abro el archivo:
    x, sr = librosa.core.load(filename,sr=None)

    #Calculo MFCCs
    features = librosa.feature.mfcc(x,sr=sr,n_mfcc=n_mfcc)
    feat_names = ['mfcc_{}'.format(i) for i in range(n_mfcc)]
    #Calculo energia:
    if energy:
        energy = librosa.feature.rms(x)
        features = np.concatenate([features,energy])
        feat_names = feat_names + ['energy']
    #Aplico media y desvio estandar por defecto
    summary_features = np.concatenate([fn(features,axis=1) for fn in summary_fn])
    feat_names = ['{}_{}'.format(name_i,summ_i) for summ_i in summary_names for name_i in feat_names]

    #Lo mismo con los delta
    if delta:
        deltafeatures = np.diff(features)
        summary_features = np.concatenate([summary_features,np.concatenate([fn(deltafeatures,axis=1) for fn in summary_fn])])
        d_names = ['d{}'.format(name) for name in feat_names]
    else:
        d_names = []

    #Y con los delta de segundo orden
    if deltadelta:
        deltadeltafeatures = np.diff(features,n=2)
        summary_features = np.concatenate([summary_features,np.concatenate([fn(deltadeltafeatures,axis=1) for fn in summary_fn])]) 
        dd_names = ['dd{}'.format(name) for name in feat_names]
    else:
        dd_names = []

    feat_names = feat_names + d_names + dd_names

    return summary_features, feat_names  

In [55]:
audio_df = pd.DataFrame()
k=0

for key in audio_files:
    print(key)
    j=0
    for audio in audio_files[key]:
        feat, names = calculate_features(audio)
        audio_row = dict(zip(names, feat))
        audio_row['digit'] = key
        audio_row['filename'] = audio
        audio_df = audio_df.append(audio_row, ignore_index=True)
        k+=1
        j+=1
    #audio_df = pd.DataFrame(audio_df)
    print(f'Procesados {j} audios en {key}.')
print(f'Procesados {k} audios.')

zero
Procesados 53 audios en zero.
one
Procesados 51 audios en one.
two
Procesados 52 audios en two.
three
Procesados 49 audios en three.
four
Procesados 49 audios en four.
five
Procesados 49 audios en five.
six
Procesados 50 audios en six.
seven
Procesados 51 audios en seven.
eight
Procesados 49 audios en eight.
nine
Procesados 49 audios en nine.
Procesados 502 audios.


In [56]:
audio_df

,ddenergy_mean,ddenergy_std,ddmfcc_0_mean,ddmfcc_0_std,ddmfcc_10_mean,ddmfcc_10_std,ddmfcc_11_mean,ddmfcc_11_std,ddmfcc_1_mean,ddmfcc_1_std,...,mfcc_5_mean,mfcc_5_std,mfcc_6_mean,mfcc_6_std,mfcc_7_mean,mfcc_7_std,mfcc_8_mean,mfcc_8_std,mfcc_9_mean,mfcc_9_std
0,6.631199e-05,0.004150,-0.990160,27.788267,0.153122,6.530779,-0.121697,5.139882,0.413138,21.068680,...,9.918743,12.048921,4.123312,5.742887,1.853000,13.878942,-12.008492,12.029095,-0.861103,8.643719
1,5.642764e-06,0.003857,0.585283,27.026859,-0.138353,6.504038,-0.102408,5.289505,-1.082853,23.488014,...,9.382134,11.625336,6.412999,7.269559,6.036138,13.719712,-7.419434,13.842501,-2.476270,6.534670
2,5.775853e-05,0.004355,0.041281,26.868860,-0.061675,6.255001,-0.211596,6.187283,-0.049250,23.516134,...,9.984041,13.456739,6.705077,6.406113,1.161983,16.052532,-8.916787,12.887153,-2.844926,8.014687
3,-2.252265e-06,0.003529,-0.201754,23.381018,-0.030292,6.520376,0.016522,5.673583,0.018416,17.848589,...,12.369701,14.574608,1.905533,5.521515,-1.570984,12.107710,-10.502273,9.074048,-3.999982,7.416301
4,-3.332240e-07,0.004071,-0.181494,17.041660,0.140187,7.650919,0.169983,5.300046,0.091065,14.404129,...,11.971924,13.733066,-3.046171,6.214321,1.984977,11.120660,-13.601109,11.414150,-2.441449,8.491489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,-2.743416e-06,0.005090,-0.047497,22.341274,-0.216634,3.746786,-0.022651,4.440460,0.004616,12.505112,...,-5.715170,21.697515,4.871535,9.140237,-10.431607,15.958239,-2.346928,9.334397,7.290930,9.416934
498,4.758083e-07,0.003703,-0.297814,23.851023,-0.096750,4.749322,0.000166,4.959522,-0.174931,12.162763,...,-7.339017,18.034012,5.381825,9.518994,-13.129568,17.599476,-4.578354,9.380426,9.121109,7.012161
499,1.378407e-06,0.002434,-0.278441,20.807295,-0.016545,5.293029,-0.039727,4.224662,-0.000419,14.383782,...,-5.905046,18.806023,5.132037,8.991982,-11.126146,18.652073,-6.921548,11.482219,5.845997,4.554769
500,-3.159247e-06,0.003777,-0.377506,23.619884,0.140942,3.841938,0.081115,5.827608,-0.094523,14.586057,...,-4.339839,19.437260,2.018462,8.253238,-10.301983,16.521338,-7.658100,10.572903,5.530354,7.121448


In [57]:
from pathlib import Path

# Guardamos nuestro dataframe en un csv
Path(f"./content").mkdir(parents=True, exist_ok=True)
audio_df.to_csv(f'./content/{directorio}.csv')